# Clean the data

In [2]:
import pandas as pd
import numpy as np
raw = pd.read_csv('df_monthly.csv')
raw.head()

,year,month,date,state,county,county_code,monthly_adj_deaths,monthly_crude_rate_10k
0,2018,1,2018-01-01,OK,Adair,40001,16,7.245720
1,2018,2,2018-02-01,OK,Adair,40001,20,9.057151
2,2018,3,2018-03-01,OK,Adair,40001,15,6.792863
3,2018,4,2018-04-01,OK,Adair,40001,14,6.340005
4,2018,5,2018-05-01,OK,Adair,40001,1,0.452858


In [3]:
# number of data points
print('number of data points:', raw.shape[0])
# number of counties
print('number of counties:', raw['county'].nunique())

number of data points: 15888
number of counties: 323


In [4]:
# convert features into correct dtype
print('data type of each feature before converting:\n',raw.dtypes)
raw[['state','county']] = raw[['state','county']].astype('string')
raw[['date']] = raw[['date']].astype('datetime64')
print('data type of each feature after converting:\n',raw.dtypes)

data type of each feature before converting:
 year                        int64
month                       int64
date                       object
state                      object
county                     object
county_code                 int64
monthly_adj_deaths          int64
monthly_crude_rate_10k    float64
dtype: object
data type of each feature after converting:
 year                               int64
month                              int64
date                      datetime64[ns]
state                             string
county                            string
county_code                        int64
monthly_adj_deaths                 int64
monthly_crude_rate_10k           float64
dtype: object


In [5]:
# rename feature names for simplicity
raw.rename(columns = {'monthly_adj_deaths':'death','monthly_crude_rate_10k':'death_rate'}, inplace=True)
# print new feature names
print(raw.columns.tolist())

['year', 'month', 'date', 'state', 'county', 'county_code', 'death', 'death_rate']


In [6]:
# sort 'raw' by 'county_code' and 'year'
raw = raw.sort_values(by=['county_code','year'])
raw = raw.reset_index(drop=True)

In [7]:
print(raw.iloc[47,:])

year                          2021
month                           12
date           2021-12-01 00:00:00
state                           OK
county                       Adair
county_code                  40001
death                           16
death_rate                8.241475
Name: 47, dtype: object


# LSTM Model

## Data Preparation

In [8]:
# scale the 'death' before feeding it into LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
minmaxscaler = MinMaxScaler()
standardscaler = StandardScaler()
raw['death_minmax'] = minmaxscaler.fit_transform(raw[['death']])
raw['death_standard'] = standardscaler.fit_transform(raw[['death']])
# print first five rows to check scaling
print(raw.head())

   year  month       date state county  county_code  death  death_rate  \
0  2018      1 2018-01-01    OK  Adair        40001     16    7.245720   
1  2018      2 2018-02-01    OK  Adair        40001     20    9.057151   
2  2018      3 2018-03-01    OK  Adair        40001     15    6.792863   
3  2018      4 2018-04-01    OK  Adair        40001     14    6.340005   
4  2018      5 2018-05-01    OK  Adair        40001      1    0.452858   

   death_minmax  death_standard  
0      0.008663       -0.181936  
1      0.010828       -0.146345  
2      0.008121       -0.190834  
3      0.007580       -0.199732  
4      0.000541       -0.315402  


In [9]:
# use onehot encoding to encode the feature 'county'
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(sparse_output = False)
county_code_onehot = onehot.fit_transform(raw[['county_code']])
print(county_code_onehot.shape)

(15888, 331)


In [10]:
# split the 'raw' data into training set (2018.01 - 2021.11) and testing set (2021.09-2021.12)
# set the start and end dates for training set
start_date_train_lstm = pd.to_datetime('2018-01-01')
end_date_train_lstm = pd.to_datetime('2021-11-01')
raw_train_lstm = raw[(raw['date'] >= start_date_train_lstm) & (raw['date'] <= end_date_train_lstm)]
print(len(raw_train_lstm) == (12*4-1)*county_code_onehot.shape[1])

#set the start and end dates for testing set
start_date_test_lstm = pd.to_datetime('2021-09-01')
end_date_test_lstm = pd.to_datetime('2021-12-01')
raw_test_lstm = raw[(raw['date'] >= start_date_test_lstm) & (raw['date'] <= end_date_test_lstm)]
print(len(raw_test_lstm) == 4*county_code_onehot.shape[1])

#convert the training set and testing set into numpy array of shape (row, column)
death_minmax_train_lstm = np.array(raw_train_lstm['death_minmax'])
death_minmax_train_lstm = death_minmax_train_lstm.reshape(-1,1)
print('death_minmax_train_lstm.shape:',death_minmax_train_lstm.shape)
death_minmax_test_lstm = np.array(raw_test_lstm['death_minmax'])
death_minmax_test_lstm = death_minmax_test_lstm.reshape(-1,1)
print('death_minmax_test_lstm.shape:',death_minmax_test_lstm.shape)

death_standard_train_lstm = np.array(raw_train_lstm['death_minmax'])
death_standard_train_lstm = death_standard_train_lstm.reshape(-1,1)
print('death_standard_train_lstm.shape:',death_standard_train_lstm.shape)
death_standard_test_lstm = np.array(raw_test_lstm['death_minmax'])
death_standard_test_lstm = death_standard_test_lstm.reshape(-1,1)
print('death_standard_test_lstm.shape:',death_standard_test_lstm.shape)

True
True
death_minmax_train_lstm.shape: (15557, 1)
death_minmax_test_lstm.shape: (1324, 1)
death_standard_train_lstm.shape: (15557, 1)
death_standard_test_lstm.shape: (1324, 1)


In [11]:
# given a county_code, find its onehot vector representation
def get_county_code_encoding(county_code):
    county_code_list = raw['county_code'].tolist()
    if county_code in county_code_list:
        county_index = county_code_list.index(county_code)
        county_encoding = county_code_onehot[county_index]
        return county_encoding
    else:
        print('County code not found.')
        return None

In [12]:
# create the county feature for training set and testing set
county_code_train_lstm = raw_train_lstm['county_code']
county_code_onehot_train_lstm = county_code_train_lstm.map(get_county_code_encoding)
county_code_onehot_train_lstm = np.vstack(county_code_onehot_train_lstm)
print('county_code_onehot_train_lstm.shape:', county_code_onehot_train_lstm.shape)

county_code_test_lstm = raw_test_lstm['county_code']
county_code_onehot_test_lstm = county_code_test_lstm.map(get_county_code_encoding)
county_code_onehot_test_lstm = np.vstack(county_code_onehot_test_lstm)
print('county_code_onehot_test_lstm.shape:', county_code_onehot_test_lstm.shape)

county_code_onehot_train_lstm.shape: (15557, 331)
county_code_onehot_test_lstm.shape: (1324, 331)


In [16]:
# horizontally stack the county feature and the death feature
data_minmax_train = np.hstack((county_code_onehot_train_lstm,death_minmax_train_lstm))
print('data_minmax_train.shape:',data_minmax_train.shape)
data_minmax_test = np.hstack((county_code_onehot_test_lstm,death_minmax_test_lstm))
print('data_minmax_test.shape:',data_minmax_test.shape)

data_standard_train = np.hstack((county_code_onehot_train_lstm,death_standard_train_lstm))
print('data_standard_train.shape:',data_standard_train.shape)
data_standard_test = np.hstack((county_code_onehot_test_lstm,death_standard_test_lstm))
print('data_standard_test.shape:',data_standard_test.shape)

data_minmax_train.shape: (15557, 332)
data_minmax_test.shape: (1324, 332)
data_standard_train.shape: (15557, 332)
data_standard_test.shape: (1324, 332)


In [17]:
# split a multivariate sequence into samples
def split_sequence(sequence, n_timestep): # n_timestep is the window size
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i+ n_timestep # find the end of the pattern
        if end_ix > len(sequence)-1: # check if we are beyond the dataset
            break
        seq_x = sequence[i:end_ix, :]
        seq_y = sequence[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [18]:
# given a two dimensional array of each county and death number, calcaute a sequence for each county and concatenate them together
def get_input_sequence(array, n_timestep):
    X_list, y_list = list(), list() #empty list to hold the sequence of each county
    for i in range(raw['county_code'].nunique()):
        n_obs_each_county = int(len(array)/raw['county_code'].nunique()) # number of observations for each county
        begin_index = n_obs_each_county*i
        end_index = begin_index + n_obs_each_county
        subset_array = array[begin_index:end_index,] # split the array into each county to create sequence for each county
        X_county, y_county = split_sequence(subset_array, n_timestep)
        X_list.append(X_county)
        y_list.append(y_county)
    # vertically stack X_county
    X = np.vstack(X_list)
    # concatenate y_county
    y = np.concatenate(y_list)
    return X, y     

In [19]:
#If X_train has shape (num_samples, n_timestep, n_features), 
#then X_test should also have shape (num_test_samples, n_timestep, n_features)**
X_minmax_train_lstm, y_minmax_train_lstm = get_input_sequence(array = data_minmax_train, n_timestep = 3)
print('X_minmax_train_lstm.shape:',X_minmax_train_lstm.shape)
print('y_minmax_train_lstm.shape:',y_minmax_train_lstm.shape)
X_minmax_test_lstm, y_minmax_test_lstm = get_input_sequence(array = data_minmax_test, n_timestep = 3)
print('X_minmax_test_lstm.shape:', X_minmax_test_lstm.shape)
print('y_minmax_test_lstm.shape:', y_minmax_test_lstm.shape)
print('')

X_standard_train_lstm, y_standard_train_lstm = get_input_sequence(array = data_standard_train, n_timestep = 3)
print('X_standard_train_lstm.shape:',X_standard_train_lstm.shape)
print('y_standard_train_lstm.shape:',y_standard_train_lstm.shape)
X_standard_test_lstm, y_standard_test_lstm = get_input_sequence(array = data_standard_test, n_timestep = 3)
print('X_standard_test_lstm.shape:', X_standard_test_lstm.shape)
print('y_standard_test_lstm.shape:', y_standard_test_lstm.shape)

X_minmax_train_lstm.shape: (14564, 3, 332)
y_minmax_train_lstm.shape: (14564,)
X_minmax_test_lstm.shape: (331, 3, 332)
y_minmax_test_lstm.shape: (331,)

X_standard_train_lstm.shape: (14564, 3, 332)
y_standard_train_lstm.shape: (14564,)
X_standard_test_lstm.shape: (331, 3, 332)
y_standard_test_lstm.shape: (331,)


In [20]:
n_timestep = X_minmax_train_lstm.shape[1]
n_features = X_minmax_train_lstm.shape[2]
print(n_timestep, n_features)

3 332


## Fit LSTM model

In [18]:
from keras_tuner import HyperModel
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.optimizers.legacy import Adam

class LSTMHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.Sequential()

        # Use for loop to add LSTM layers according to the choice
        for i in range(hp.Int('n_layers', 1, 6)):  # Here, we are allowing 1 to 3 LSTM layers
            if i == 0:
                # first layer specifies input_shape and returns sequences if there are more layers to follow
                model.add(layers.LSTM(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                                      activation='relu',
                                      input_shape=(self.input_shape), 
                                      return_sequences=True if hp.Int('n_layers', 1, 3) > 1 else False)) 
            else:
                # subsequent layers return sequences if they are not the last one
                model.add(layers.LSTM(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                                      activation='relu',
                                      return_sequences=True if i < hp.Int('n_layers', 1, 3) - 1 else False)) 

        model.add(layers.Dense(1,))

        model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])),
                      loss='mse')

        return model

### LSTM MinMax Scaler

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import random
import tensorflow as tf

input_shape = (n_timestep, n_features)

hypermodel = LSTMHyperModel(input_shape)

tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=30,  # total number of hypter-parameter combinations
    executions_per_trial=3, # for each combination, how many trails to traint to average the result
    directory='random_search_lstm',
    project_name='tuning_lstm'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10) # the validation loss does not improve for 10 epochs

random.seed(9999)
np.random.seed(9999)
tf.random.set_seed(9999)

tuner.search(X_minmax_train_lstm, y_minmax_train_lstm, 
             epochs=50,
             validation_split=0.2,  # or use a separate validation set
             callbacks=[early_stopping]) #This callback will be applied to each trial of the hyperparameter search

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it on the data
best_lstm_model = tuner.hypermodel.build(best_hps)

# Build the model with the optimal hyperparameters and train it on the data
best_lstm_model = tuner.hypermodel.build(best_hps)

# Fit the model on the entire training set
early_stopping = EarlyStopping(monitor='loss', patience=10)
best_lstm_model.fit(X_minmax_train_lstm, y_minmax_train_lstm, epochs=50, callbacks=[early_stopping], verbose=0)
best_lstm_model.summary()

INFO:tensorflow:Reloading Tuner from random_search_lstm/tuning_lstm/tuner0.json
INFO:tensorflow:Oracle triggered exit
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 512)               1730560   
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 1731073 (6.60 MB)
Trainable params: 1731073 (6.60 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [64]:
# calcuate the MSE for training set and testing set on scaled data
mse_minmax_train_lstm_scaled = best_lstm_model.evaluate(X_minmax_train_lstm, y_minmax_train_lstm)
print('mse_minmax_train_lstm_scaled:', mse_minmax_train_lstm_scaled)
mse_minmax_test_lstm_scaled = best_lstm_model.evaluate(X_minmax_test_lstm, y_minmax_test_lstm)
print('mse_minmax_test_lstm_scaled:', mse_minmax_test_lstm_scaled)

456/456 [==============================] - 1s 2ms/step - loss: 3.1786e-05
mse_minmax_train_lstm_scaled: 3.178596671205014e-05
11/11 [==============================] - 0s 3ms/step - loss: 4.2090e-05
mse_minmax_test_lstm_scaled: 4.209001417621039e-05


__*We can exclude the possibility of overfitting*__

In [49]:
# Calculate MSE of the test set on the orignal scale
from sklearn.metrics import mean_squared_error
y_predicted_minmax_test_lstm = minmaxscaler.inverse_transform(best_lstm_model.predict(X_minmax_test_lstm))
y_predicted_minmax_test_lstm = y_predicted_minmax_test_lstm.reshape(y_predicted_minmax_test_lstm.shape[0])
y_true_test = raw.loc[(raw['month'] == 12) & (raw['year'] == 2021)]['death'].values
mse_standard_test_lstm = mean_squared_error(y_true_test, y_predicted_minmax_test_lstm)
print('mse_minmax_test_lstm =', mse_minmax_test_lstm) #unbiased point estimator

11/11 [==============================] - 0s 3ms/step
mse_minmax_test_lstm = 13572.338563612746


In [50]:
print(min(y_predicted_minmax_test_lstm))
print(min(y_true_test))
print(max(y_predicted_minmax_test_lstm))
print(max(y_true_test))

2.6933835
0
1333.1964
1399


### LSTM Standard Scaler

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
import random
import tensorflow as tf

input_shape = (n_timestep, n_features)

hypermodel = LSTMHyperModel(input_shape)

tuner_std = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=30,  # total number of hypter-parameter combinations
    executions_per_trial=3, # for each combination, how many trails to traint to average the result
    directory='random_search_lstm_std',
    project_name='tuning_lstm_std'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=10) # the validation loss does not improve for 10 epochs

random.seed(9999)
np.random.seed(9999)
tf.random.set_seed(9999)

tuner_std.search(X_standard_train_lstm, y_standard_train_lstm, 
             epochs=50,
             validation_split=0.2,  # or use a separate validation set
             callbacks=[early_stopping]) #This callback will be applied to each trial of the hyperparameter search

# Get the optimal hyperparameters
best_hps_std = tuner_std.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it on the data
best_lstm_model_std = tuner_std.hypermodel.build(best_hps_std)

# Fit the model on the entire training set
early_stopping = EarlyStopping(monitor='loss', patience=10)
best_lstm_model_std.fit(X_standard_train_lstm, y_standard_train_lstm, epochs=50, callbacks=[early_stopping], verbose=0)
best_lstm_model_std.summary()

Trial 30 Complete [00h 13m 57s]
val_loss: 0.0018580974234888952

Best val_loss So Far: 0.0008871587439595411
Total elapsed time: 03h 07m 53s
INFO:tensorflow:Oracle triggered exit
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 3, 384)            1101312   
                                                                 
 lstm_5 (LSTM)               (None, 3, 224)            545664    
                                                                 
 lstm_6 (LSTM)               (None, 3, 384)            935424    
                                                                 
 lstm_7 (LSTM)               (None, 3, 416)            1332864   
                                                                 
 lstm_8 (LSTM)               (None, 64)                123136    
                                                                 
 dense_

In [63]:
# calcuate the MSE for training set and testing set on scaled data
mse_standard_train_lstm_scaled = best_lstm_model_std.evaluate(X_standard_train_lstm, y_standard_train_lstm)
print('mse_standard_train_lstm_scaled:', mse_standard_train_lstm_scaled)
mse_standard_test_lstm_scaled = best_lstm_model_std.evaluate(X_standard_test_lstm, y_standard_test_lstm)
print('mse_standard_test_lstm_scaled:', mse_standard_test_lstm_scaled)

456/456 [==============================] - 3s 7ms/step - loss: 2.8821e-05
mse_standard_train_lstm_scaled: 2.882141234294977e-05
11/11 [==============================] - 0s 7ms/step - loss: 3.2378e-05
mse_standard_test_lstm_scaled: 3.237785131204873e-05


__*We can exclude the possibility of overfitting*__

In [43]:
# Calculate MSE of the test set on the orignal scale
from sklearn.metrics import mean_squared_error
y_predicted_standard_test_lstm = standardscaler.inverse_transform(best_lstm_model_std.predict(X_standard_test_lstm))
y_predicted_standard_test_lstm = y_predicted_standard_test_lstm.reshape(y_predicted_standard_test_lstm.shape[0])
y_true_test = raw.loc[(raw['month'] == 12) & (raw['year'] == 2021)]['death'].values
mse_standard_test_lstm = mean_squared_error(y_true_test, y_predicted_standard_test_lstm)
print('mse_standard_test_lstm =', mse_standard_test_lstm) #unbiased point estimator

11/11 [==============================] - 0s 7ms/step
mse_standard_test_lstm = 12002.308474647894


In [51]:
print(min(y_predicted_standard_test_lstm))
print(min(y_true_test))
print(max(y_predicted_standard_test_lstm))
print(max(y_true_test))

36.3917
0
117.51195
1399


# ARIMAX Model

## Data Preparation

In [21]:
# split the 'raw' data into training set (2018.01 - 2021.11) and testing set (2021.12)
# set the start and end dates for training set
start_date_train_arimax = pd.to_datetime('2018-01-01')
end_date_train_arimax = pd.to_datetime('2021-11-01')
raw_train_arimax = raw[(raw['date'] >= start_date_train_arimax) & (raw['date'] <= end_date_train_arimax)]
print(len(raw_train_arimax) == (12*4-1)*county_code_onehot.shape[1])

#set the date for testing set
date_test_arimax = pd.to_datetime('2021-12-01')
raw_test_arimax = raw[(raw['date'] == date_test_arimax)]
print(len(raw_test_arimax) == 1*county_code_onehot.shape[1])

True
True


In [22]:
# create training data and testing data for 'death'
death_train_arimax = raw_train_arimax['death'].to_numpy()
print('death_train_arimax.shape:',death_train_arimax.shape)
death_test_arimax = raw_test_arimax['death'].to_numpy()
print('death_test_arimax.shape:',death_test_arimax.shape)

death_train_arimax.shape: (15557,)
death_test_arimax.shape: (331,)


In [25]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
county_code_pca = pca.fit_transform(county_code_onehot)

In [32]:
# given a county_code, find its pca representation
def get_county_code_pca(county_code):
    county_code_list = raw['county_code'].tolist()
    if county_code in county_code_list:
        county_index = county_code_list.index(county_code)
        county_encoding = county_code_pca[county_index]
        return county_encoding
    else:
        print('County code not found.')
        return None

In [33]:
# create training data and testing data for county names
county_code_train_arimax = raw_train_arimax['county_code']
county_code_pca_train_arimax = county_code_train_arimax.map(get_county_code_pca)
county_code_pca_train_arimax = np.vstack(county_code_pca_train_arimax)
print('county_code_pca_train_arimax.shape:', county_code_pca_train_arimax.shape)

county_code_test_arimax = raw_test_arimax['county_code']
county_code_pca_test_arimax = county_code_test_arimax.map(get_county_code_pca)
county_code_pca_test_arimax = np.vstack(county_code_pca_test_arimax)
print('county_code_pca_test_arimax.shape:', county_code_pca_test_arimax.shape)

county_code_pca_train_arimax.shape: (15557, 2)
county_code_pca_test_arimax.shape: (331, 2)


## Fit ARIMAX model

In [38]:
from statsmodels.tsa.arima.model import ARIMA
import itertools
import warnings

# Find the best values of p, d, q
p = d = q = range(0, 4) #take any value between 0 and 3
pdq = list(itertools.product(p, d, q)) # Generate all different combinations of p, d and q triplets

arimax_model_results = []
# Find the best parameters
for parms in pdq: 
    try:
        model_arimax = ARIMA(death_train_arimax, order = parms, exog = county_code_pca_train_arimax)
        model_arimax_fit = model_arimax.fit()
        arimax_model_results.append((parms, model_arimax_fit.aic))
        print(f"ARIMA model with parameters {parms} successfully fitted. AIC: {model_arimax_fit.aic}")
        
    except:
        print(f"ARIMA model with parameters {parms} failed to fit. Error: {e}")
        continue
best_params = min(arimax_model_results, key = lambda x:x[1])
print(f"The best ARIMAX parameters are: p={best_params[0][0]}, d={best_params[0][1]}, q={best_params[0][2]} with AIC={best_params[1]}")

ARIMA model with parameters (0, 0, 0) successfully fitted. AIC: 191026.03652221922
ARIMA model with parameters (0, 0, 1) successfully fitted. AIC: 175864.82341712218
ARIMA model with parameters (0, 0, 2) successfully fitted. AIC: 166716.19832595246
ARIMA model with parameters (0, 0, 3) successfully fitted. AIC: 162108.52324548084
ARIMA model with parameters (0, 1, 0) successfully fitted. AIC: 150162.1990181963
ARIMA model with parameters (0, 1, 1) successfully fitted. AIC: 149635.71695110365
ARIMA model with parameters (0, 1, 2) successfully fitted. AIC: 149637.69721823608
ARIMA model with parameters (0, 1, 3) successfully fitted. AIC: 149614.01264071662
ARIMA model with parameters (0, 2, 0) successfully fitted. AIC: 163521.09508376668
ARIMA model with parameters (0, 2, 1) successfully fitted. AIC: 150165.99257663864
ARIMA model with parameters (0, 2, 2) successfully fitted. AIC: 149640.19726717687


/Users/jasonding/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


ARIMA model with parameters (0, 2, 3) successfully fitted. AIC: 149641.93474158575
ARIMA model with parameters (0, 3, 0) successfully fitted. AIC: 181428.0597283297
ARIMA model with parameters (0, 3, 1) successfully fitted. AIC: 163526.93863782496
ARIMA model with parameters (0, 3, 2) successfully fitted. AIC: 150197.94379047945
ARIMA model with parameters (0, 3, 3) successfully fitted. AIC: 149711.44501491872
ARIMA model with parameters (1, 0, 0) successfully fitted. AIC: 149899.74248491885
ARIMA model with parameters (1, 0, 1) successfully fitted. AIC: 149469.8519410194
ARIMA model with parameters (1, 0, 2) successfully fitted. AIC: 149469.4935677942
ARIMA model with parameters (1, 0, 3) successfully fitted. AIC: 149459.87924134193
ARIMA model with parameters (1, 1, 0) successfully fitted. AIC: 149646.8382117896
ARIMA model with parameters (1, 1, 1) successfully fitted. AIC: 149637.68158298437
ARIMA model with parameters (1, 1, 2) successfully fitted. AIC: 149623.37521771598
ARIMA mo

/Users/jasonding/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ARIMA model with parameters (1, 2, 3) successfully fitted. AIC: 149557.8983868864
ARIMA model with parameters (1, 3, 0) successfully fitted. AIC: 170239.8560344665
ARIMA model with parameters (1, 3, 1) successfully fitted. AIC: 157092.05131368144
ARIMA model with parameters (1, 3, 2) successfully fitted. AIC: 149712.84727037314
ARIMA model with parameters (1, 3, 3) successfully fitted. AIC: 150182.60477026924
ARIMA model with parameters (2, 0, 0) successfully fitted. AIC: 149467.65508420859
ARIMA model with parameters (2, 0, 1) successfully fitted. AIC: 149468.37609491052
ARIMA model with parameters (2, 0, 2) successfully fitted. AIC: 149384.9731582981
ARIMA model with parameters (2, 0, 3) successfully fitted. AIC: 149460.93966323347
ARIMA model with parameters (2, 1, 0) successfully fitted. AIC: 149642.4215351455


...

ARIMA model with parameters (3, 3, 1) successfully fitted. AIC: 153700.90669278803


/Users/jasonding/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ARIMA model with parameters (3, 3, 2) successfully fitted. AIC: 152790.7013755003


/Users/jasonding/anaconda3/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ARIMA model with parameters (3, 3, 3) successfully fitted. AIC: 149686.44727607747
The best ARIMAX parameters are: p=2, d=0, q=2 with AIC=149384.9731582981


In [43]:
# Choose the ARIMAX model with best parameters (2,0,2)
best_arimax_model = ARIMA(death_train_arimax, exog = county_code_pca_train_arimax, order = (2,0,2))
best_arimax_model = best_arimax_model.fit()
print("Best ARIMAX model AIC: ", best_arimax_model.aic)
print("Best ARIMAX model BIC: ", best_arimax_model.bic)

Best ARIMAX model AIC:  149384.9731582981
Best ARIMAX model BIC:  149446.19128611477


In [50]:
# Predict using the best ARIMAX model
arimax_test_predicted = []
for i in range(len(county_code_pca_test_arimax)):
    county_code_pca_test_arimax_single_county = county_code_pca_test_arimax[i,:].reshape(1,-1)
    arimax_test_predicted_single_county = best_arimax_model.get_forecast(steps = 1, exog = county_code_pca_test_arimax_single_county)
    arimax_test_predicted_single_county = arimax_test_predicted_single_county.predicted_mean[0]
    arimax_test_predicted.append(arimax_best_predicted_single_county)

# calculate the MSE on testing set
from sklearn.metrics import mean_squared_error
mse_test_arimax = mean_squared_error(death_test_arimax, arimax_test_predicted)
print('MSE of prediction from best ARIMAX model (2,0,2) on testing data =', mse_test_arimax)

MSE of prediction from best ARIMAX model (2,0,2) on testing data = 15013.826618600395


In [51]:
# Predict using the best ARIMAX model on training data
arimax_train_predicted = best_arimax_model.predict(start=0, end=len(death_train_arimax)-1, exog=county_code_pca_train_arimax)

# Calculate the MSE on training data
mse_train_arimax = mean_squared_error(death_train_arimax, arimax_train_predicted)
print('MSE of prediction from best ARIMAX model (2,0,2) on training data =', mse_train_arimax)

MSE of prediction from best ARIMAX model (2,0,2) on training data = 865.5383232344526


# Exponential Smoothing

In [90]:
raw_exp = raw.copy()
# Convert 'date' to datetime format
raw_exp['date'] = pd.to_datetime(raw_exp['date'])
# Set 'date' as the index
raw_exp = raw.set_index('date')
# Sort the DataFrame by the index (date)
raw_exp = raw_exp.sort_index()

In [91]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

exp_predicted = {}
# loop over each county_code:
for county_code in raw_exp['county_code'].unique():
    df_county_code = raw_exp[raw_exp['county_code'] == county_code]['death']
    df_county_code.index = pd.date_range(start='2018-01-01', end='2021-12-31', freq='M')
    df_county_code = df_county_code.sort_index()  # sort dataframe in ascending order of date
    df_county_code.index = df_county_code.index.to_period('M')  # set the frequency to monthly

    # Split into training and test data
    exp_train = df_county_code[:-1]  # all data up to November 2021
    exp_test = df_county_code[-1:]   # December 2021
    exp_model = ExponentialSmoothing(exp_train, seasonal_periods=12, trend='add', seasonal='add').fit()
    # Predict for December 2021
    exp_predicted_dec = exp_model.predict(start=len(exp_train), end=len(exp_train)) # len(train) corresponds to the next time step right after the end of the training data
    exp_predicted[county_code] = exp_predicted_dec

# sort the exp_predicted dictionary by the value of the key
exp_predicted = dict(sorted(exp_predicted.items()))
for key in exp_predicted:
    exp_predicted[key] = exp_predicted[key].values[0]

In [103]:
# extract the true value of death in Dec 2021 for each county and store them in a dictionary
raw_exp_sorted = raw_exp.sort_values('county_code')
raw_exp_test = raw_exp_sorted[raw_exp_sorted.index == '2021-12-01']
exp_test_true = {}
for i in range(len(raw_exp_test)):
            exp_test_true[raw_exp_test['county_code'].iloc[i]] = raw_exp_test['death'].iloc[i]

In [111]:
from sklearn.metrics import mean_squared_error
mse_exp = mean_squared_error(list(exp_test_true.values()), list(exp_predicted.values()))
print('MSE of prediction from triple exponential smoothing model =', mse_exp)

MSE of prediction from triple exponential smoothing model = 155.42821982122163


In [112]:
for i, j in zip(list(exp_test_true.values()), list(exp_predicted.values())):
    print(i,j)

16 19.9723761749089
0 0.6651774548329896
13 10.526315406745335
0 0.017434120834306677
17 19.104696961356474
12 3.789342416418102
30 32.280697851459195
28 16.403510296867353
59 58.368875065857814
37 39.05644792818512
26 28.70174328750262
13 16.10513289059496
0 0.1580994164426947
109 117.73661085056638
1 0.9824548825437034
82 71.12956397014455
1 4.210525634906256
13 9.876964859859235
49 38.24559425002614
31 16.91215234570568
35 29.59656672470758
0 4.192981611839382
1 0.561402739637874
54 30.17545290632733
20 20.5934052067256
31 26.333395630920545
0 1.397499354339277
0 5.947368610571766
0 0.5438606292221699
0 0.4557439582039453
0 5.931172196221459
12 9.1227865193279
13 16.140271274479357
0 0.4035072073015743
1 6.719306806364809
40 30.438678097075528
1 12.63143652663782
12 8.824553361648977
0 0.7368408908658062
39 37.92963256897182
20 20.561722854455137
32 16.315778607075956
9 5.701753419361289
22 19.85991127483378
21 30.070114353799994
16 21.350571792652776
10 -0.17463873514433947
12 8.56